In [1]:
# Author  : Karthikeyan Rasipalayam Durairaj 
# Dataset : HTTP requests to the NASA Kennedy Space Center—is freely available 
#/FileStore/tables/NASA_access_log_Jul95.gz

In [2]:
# import packages
from pyspark.sql import SparkSession, HiveContext, SQLContext
from pyspark.sql.functions import when, desc
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import to_date, trunc, unix_timestamp, date_format
from pyspark.sql.window import Window
from datetime import datetime
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import regexp_extract


In [3]:
base_df = spark.read.text("dbfs:/FileStore/tables/access_log_Jul95_sample-03e7f.txt")


In [4]:
base_df.collect()

Out[ 38 ]: 
[Row(value='199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0'),
 Row(value='205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985'),
 Row(value='d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 200 7074'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /shuttle/countdown/count.gif HTTP/1.0" 200 40310'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /images/KSC-logosmall.gif HTTP/1.0" 200 1204'),
 Row(value='d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /shuttle/countdown/count.gif HTTP/1.0" 200 40310'),
 Row(value='d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786'),
 Row(value='d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /images/KSC-logosmall.gif HTTP/1.0" 200 1204'),
 Row(value='129.94.144.152 - - [01/Jul/1995:00:00:17 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:17 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713'),
 Row(value='ppptky391.asahi-net.or.jp - - [01/Jul/1995:00:00:18 -0400] "GET /facts/about_ksc.html HTTP/1.0" 200 3977'),
 Row(value='net-1-141.eden.com - - [01/Jul/1995:00:00:19 -0400] "GET /shuttle/missions/sts-71/images/KSC-95EC-0916.jpg HTTP/1.0" 200 34029'),
 Row(value='ppptky391.asahi-net.or.jp - - [01/Jul/1995:00:00:19 -0400] "GET /images/launchpalms-small.gif HTTP/1.0" 200 11473'),
 Row(value='205.189.154.54 - - [01/Jul/1995:00:00:24 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='waters-gw.starway.net.au - - [01/Jul/1995:00:00:25 -0400] "GET /shuttle/missions/51-l/mission-51-l.html HTTP/1.0" 200 6723'),
 Row(value='ppp-mia-30.shadow.net - - [01/Jul/1995:00:00:27 -0400] "GET / HTTP/1.0" 200 7074'),
 Row(value='205.189.154.54 - - [01/Jul/1995:00:00:29 -0400] "GET /shuttle/countdown/count.gif HTTP/1.0" 200 40310'),
 Row(value='alyssa.prodigy.com - - [01/Jul/1995:00:00:33 -0400] "GET /shuttle/missions/sts-71/sts-71-patch-small.gif HTTP/1.0" 200 12054'),
 Row(value='ppp-mia-30.shadow.net - - [01/Jul/1995:00:00:35 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 200 5866'),
 Row(value='dial22.lloyd.com - - [01/Jul/1995:00:00:37 -0400] "GET /shuttle/missions/sts-71/images/KSC-95EC-0613.jpg HTTP/1.0" 200 61716'),
 Row(value='smyth-pc.moorecap.com - - [01/Jul/1995:00:00:38 -0400] "GET /history/apollo/apollo-13/images/70HC314.GIF HTTP/1.0" 200 101267'),
 Row(value='205.189.154.54 - - [01/Jul/1995:00:00:40 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786'),
 Row(value='ix-orl2-01.ix.netcom.com - - [01/Jul/1995:00:00:41 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='ppp-mia-30.shadow.net - - [01/Jul/1995:00:00:41 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786'),
 Row(value='ppp-mia-30.shadow.net - - [01/Jul/1995:00:00:41 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 200 363'),
 Row(value='205.189.154.54 - - [01/Jul/1995:00:00

In [5]:
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
status_pattern = r'\s(\d{3})\s'
content_size_pattern = r'\s(\d+)$'


logs_df = base_df.select(regexp_extract('value', host_pattern, 1).alias('host'),
                         regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                         regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                         regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
                         regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
                         regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                         regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))



In [6]:
logs_df.show()

+--------------------+--------------------+------+--------------------+--------+------+------------+
 host| timestamp|method| endpoint|protocol|status|content_size|
+--------------------+--------------------+------+--------------------+--------+------+------------+
 199.72.81.55|01/Jul/1995:00:00...| GET| /history/apollo/|HTTP/1.0| 200| 6245|
unicomp6.unicomp.net|01/Jul/1995:00:00...| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 199.120.110.21|01/Jul/1995:00:00...| GET|/shuttle/missions...|HTTP/1.0| 200| 4085|
 burger.letters.com|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 304| 0|
 199.120.110.21|01/Jul/1995:00:00...| GET|/shuttle/missions...|HTTP/1.0| 200| 4179|
 burger.letters.com|01/Jul/1995:00:00...| GET|/images/NASA-logo...|HTTP/1.0| 304| 0|
 burger.letters.com|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 200| 0|
 205.212.115.106|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 200| 3985|
 d104.aa.net|01/Jul/1995:00:00...| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 129.94.144.152|01/Jul/1995:00:00...| GET| /|HTTP/1.0| 200| 7074|
unicomp6.unicomp.net|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 200| 40310|
unicomp6.unicomp.net|01/Jul/1995:00:00...| GET|/images/NASA-logo...|HTTP/1.0| 200| 786|
unicomp6.unicomp.net|01/Jul/1995:00:00...| GET|/images/KSC-logos...|HTTP/1.0| 200| 1204|
 d104.aa.net|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 200| 40310|
 d104.aa.net|01/Jul/1995:00:00...| GET|/images/NASA-logo...|HTTP/1.0| 200| 786|
 d104.aa.net|01/Jul/1995:00:00...| GET|/images/KSC-logos...|HTTP/1.0| 200| 1204|
 129.94.144.152|01/Jul/1995:00:00...| GET|/images/ksclogo-m...|HTTP/1.0| 304| 0|
 199.120.110.21|01/Jul/1995:00:00...| GET|/images/launch-lo...|HTTP/1.0| 200| 1713|
ppptky391.asahi-n...|01/Jul/1995:00:00...| GET|/facts/about_ksc....|HTTP/1.0| 200| 3977|
 net-1-141.eden.com|01/Jul/1995:00:00...| GET|/shuttle/missions...|HTTP/1.0| 200| 34029|
+--------------------+--------------------+------+--------------------+--------+------+------------+
only showing top 20 rows